# Athena -> S3 data transfer

In [40]:
import boto3
import pandas as pd
import logging.config
import time
import json

In [41]:
with open(r'api_keys.txt', 'r') as f:
    data = f.readlines()
    AWS_ACCESS_KEY = str(data[0]).strip()
    AWS_SECRET_KEY = str(data[1]).strip()
    REGION_NAME = str(data[2]).strip()
    DATABASE_NAME = str(data[3]).strip()
    OUTPUT_LOCATION = str(data[4]).strip()
    S3_BUCKET_NAME = str(data[5]).strip()
    S3_OUTPUT_DIRECTORY = str(data[6]).strip()

athena_client = boto3.client(
    "athena", 
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION_NAME
)

s3_client = boto3.client(
    "s3",
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = REGION_NAME
)

In [95]:
query_response = dict()

def perform_query(sql_statement, data_name):
    global query_response
    start_time = time.time()
    # Initializing Query Execution
    response = athena_client.start_query_execution(
    QueryString = sql_statement,
    QueryExecutionContext = {"Database": DATABASE_NAME},
    ResultConfiguration = {
        "OutputLocation": OUTPUT_LOCATION,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
    )
    query_response.update({data_name: response['QueryExecutionId']})

    end_time = time.time()

    return f'Total Time taken to run query and write to S3: {end_time - start_time} seconds'

def query_results(data_name):
    global query_response
    # Obtaining query results

    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response[data_name]}.csv", data_name
        )

    return pd.read_csv(data_name)

In [2]:
# Data 1: traffic volume by label
perform_query('''
    select predicted_labels, avg(mean_trafficvolume) as "mean_traffic" 
    from data_final 
    group by predicted_labels 
    order by predicted_labels asc''', 'traffic_by_label')

# Data 2: accident volume by label
perform_query('''
    select predicted_labels, avg(mean_accidentvolume) as "mean_accident" 
    from data_final 
    group by predicted_labels 
    order by predicted_labels asc''', 'accident_by_label')

# Data 3: crashrate by label
perform_query('''
    select predicted_labels, avg(mean_crashrate) as "mean_cr" 
    from data_final 
    group by predicted_labels 
    order by predicted_labels asc''', 'crashrate_by_label')

# Get the whole data
perform_query('''
    select accident_address, accident_latitude, accident_longitude, risk_level, day_week, hour, mean_trafficvolume, mean_accidentvolume
    from data_final
    order by dr_number''', 'all_data')



# Lists the table names and their corresponding ids in S3
# print(query_response)


In [97]:
with open('s3_data_locations.json', 'w') as writeFile:
    json.dump(query_response, writeFile)

In [1]:
with open('s3_data_locations.json', 'r') as readFile:
    s = json.load(readFile)


In [167]:
df = query_results('all_data')

In [168]:
df.head()

,accident_address,accident_latitude,accident_longitude,risk_level,day_week,hour,mean_trafficvolume,mean_accidentvolume
0,"PICO, Los Angeles",34.0395,-118.2656,low,Monday,22,204447.984615,268.707692
1,"1300 N BROADWAY, Los Angeles",34.0695,-118.2324,medium,Wednesday,4,56487.460674,369.303371
2,"PICO, Los Angeles",34.0424,-118.2718,low,Sunday,1,204447.984615,268.707692
3,"FIGUEROA, Los Angeles",34.0344,-118.2714,low,Saturday,20,204447.984615,268.707692
4,"COLLEGE, Los Angeles",34.0637,-118.2361,medium,Saturday,16,56487.460674,369.303371
